<a href="https://colab.research.google.com/github/Junyoungs7/Pearson_Correlation_with_Recommend_Movie/blob/main/Movie_recommand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 수집: kaggle

In [1]:
!pip install kaggle

In [2]:
pwd

'/content'

In [3]:
import os

os.environ["KAGGLE_USERNAME"] = '`'
os.environ["KAGGLE_KEY"] = '`'
!kaggle datasets download -d rounakbanik/the-movies-dataset

 95% 217M/228M [00:00<00:00, 277MB/s]
100% 228M/228M [00:00<00:00, 278MB/s]


In [4]:
from zipfile import ZipFile

with ZipFile('/content/the-movies-dataset.zip','r') as zip:
  zip.extractall('movie-data')

# 데이터 확인

In [5]:
import pandas as pd
import numpy as np
import json

data = pd.read_csv('/content/movie-data/movies_metadata.csv')
data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
data.shape

(45466, 24)

# 데이터 전처리

In [7]:
data = data[['id','genres','original_title','original_language']]
data = data.rename(columns={'id':'movieId'})
data = data[data['original_language']=='en']
data.head()

,movieId,genres,original_title,original_language
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,en
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,en
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,en
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,en
4,11862,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,en


In [8]:
ratings = pd.read_csv('/content/movie-data/ratings_small.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [9]:
ratings.describe()

,userId,movieId,rating
count,100004.000000,100004.000000,100004.000000
mean,347.011310,12548.664363,3.543608
std,195.163838,26369.198969,1.058064
min,1.000000,1.000000,0.500000
25%,182.000000,1028.000000,3.000000
50%,367.000000,2406.500000,4.000000
75%,520.000000,5418.000000,4.000000
max,671.000000,163949.000000,5.000000


In [10]:
data.movieId = pd.to_numeric(data.movieId, errors='coerce')
# pd.to_numeric() : 문자열을 숫자 타입으로 변환
ratings.movieId = pd.to_numeric(ratings.movieId, errors='coerce')

In [11]:
def parse_genres(genres_str):
  genres = json.loads(genres_str.replace('\'','"'))
  genres_list = []
  for g in genres:
    genres_list.append(g['name'])

  return genres_list

data['genres'] = data['genres'].apply(parse_genres)
# df.apply() : 각 행에 해당 함수를 적용

In [12]:
data.head()

,movieId,genres,original_title,original_language
0,862,"[Animation, Comedy, Family]",Toy Story,en
1,8844,"[Adventure, Fantasy, Family]",Jumanji,en
2,15602,"[Romance, Comedy]",Grumpier Old Men,en
3,31357,"[Comedy, Drama, Romance]",Waiting to Exhale,en
4,11862,[Comedy],Father of the Bride Part II,en


In [13]:
data = pd.merge(ratings, data, on='movieId', how='inner')
# pd.merge() : 두 개의 데이터프레임을 병합
data.head()

,userId,movieId,rating,genres,original_title,original_language
0,1,1371,2.5,[Drama],Rocky III,en
1,4,1371,4.0,[Drama],Rocky III,en
2,7,1371,3.0,[Drama],Rocky III,en
3,19,1371,4.0,[Drama],Rocky III,en
4,21,1371,3.0,[Drama],Rocky III,en


In [14]:
# Pivot Table
matrix = data.pivot_table(index='userId', columns='original_title', values='rating')
matrix.head(5)

original_title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,...,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# 상관관계 조사(Pearson Correlation)
GENRE_WEIGHT = 0.1

def pearsonR(s1, s2):
  s1_c = s1 - s1.mean()
  s2_c = s2 - s2.mean()
  return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

def recommend(input_movie, matrix, n, similar_genre=True):
  input_genres = data[data['original_title'] == input_movie]['genres'].iloc[0]

  result = []
  for title in matrix.columns:
    if title == input_movie:
      continue

    cor = pearsonR(matrix[input_movie], matrix[title])

    if similar_genre and len(input_genres) > 0:
      temp_genres = data[data['original_title'] == title]['genres'].iloc[0]
      same_count = np.sum(np.isin(input_genres, temp_genres))
      # np.isin() : 배열을 비교하여 똑같은 요소가 있으면 true 반환
      cor += (GENRE_WEIGHT * same_count)

    if np.isnan(cor):
      continue
    else:
      result.append((title, '{:.2f}'.format(cor), temp_genres))

  result.sort(key=lambda r: r[1], reverse=True)

  return result[:n]


In [22]:
recommend_result = recommend('The Dark Knight', matrix, 10, similar_genre=True)
pd.DataFrame(recommend_result, columns=['Title', 'Correlation','Genre'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


,Title,Correlation,Genre
0,Prom Night,0.87,"[Crime, Horror]"
1,Wild Wild West,0.87,"[Action, Adventure, Comedy, Science Fiction, W..."
2,Blue Thunder,0.73,"[Science Fiction, Action, Thriller, Crime, Drama]"
3,Topaz,0.68,"[Action, Drama, Mystery, Thriller]"
4,Yamakasi - Les samouraïs des temps modernes,0.68,"[Action, Crime, Drama]"
5,Best Seller,0.67,"[Action, Crime, Drama, Thriller]"
6,Midnight in the Garden of Good and Evil,0.67,"[Crime, Drama, Mystery, Thriller]"
7,The Sentinel,0.66,"[Action, Thriller, Crime]"
8,Big Bad Mama,0.64,"[Action, Comedy, Crime, Drama]"
9,The Enforcer,0.63,"[Drama, Action, Crime]"
